# Задача

![задание](logo.png)

Весь процесс доставки — от приема в отделении до вручения получателю — состоит из большого числа операций. Отправление запаковывают, перевозят на склад и транспортируют между сортировочными пунктами. Если доставка едет из-за границы, то дополнительно появляются операции на зарубежной и российской таможне.

Ускорить и удешевить доставку помогают крупные логистические хабы. Там мелкие грузы сортируются и отправляются в соседние регионы или собираются в новые контейнеры для отправки в другие макрорегионы. Таким образом, почтовые отправления путешествуют по сети сортировочных центров, как кровь по капиллярам, и в конце концов добираются в любые точки нашей страны.

Несмотря на высокий уровень системы безопасности, по-прежнему остается риск пропаж или порчи отправлений: перемещений и операций с посылками очень много, кроме того, в процессе может сыграть человеческий фактор.

Точное предсказание пропаж и их локализация позволит повысить надежность системы — гарантировать доставку отправлений в срок и снизить расходы на транспортировку. Предлагаем участникам чемпионата решить эту задачу — разработать модель предсказания потери почтовых отправлений.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDClassifier
from scipy.stats import uniform
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFE
from sklearn import svm
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
import tensorflow as tf 
import numpy as np
import pickle
from sklearn.feature_selection import RFECV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
%matplotlib inline
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import HistGradientBoostingClassifier
import warnings
from sklearn import preprocessing
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder

2022-11-19 01:54:52.007206: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-19 01:54:52.007253: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("train_dataset_train.csv",low_memory=False)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,1043_-1,628629,Участок,7503.0,N,0.0,Y,N,87.0,5.0,1.0,0.0,2.0,0.00,0.0,"Fishhook,USB",41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,1023_-1,102976,ММПО,7503.0,N,0.0,N,N,107.0,5.0,1.0,0.0,2.0,0.00,0.0,"screen protector,Case(Q613B),case(208B40-DB)",68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,1018_-1,620962,Цех,7503.0,N,0.0,Y,N,50.0,5.0,1.0,0.0,2.0,0.00,0.0,"Pendant Necklaces,Rings for Women,Necklaces",56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,1019_-1,344964,Цех,7503.0,N,0.0,Y,N,416.0,5.0,1.0,0.0,2.0,35.34,0.0,Motorcycle Signal Lamp,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,1020_-1,629819,Участок,7503.0,N,0.0,Y,N,795.0,5.0,1.0,0.0,2.0,52.52,0.0,backpack,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,1022_-1,102976,ММПО,7503.0,N,0.0,N,N,25.0,5.0,0.0,0.0,2.0,0.00,0.0,KEY CHAIN,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,0
5999996,2234489,1022_-1,241963,Цех,7503.0,N,0.0,Y,N,83.0,5.0,1.0,0.0,2.0,0.00,0.0,Dolls,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,0
5999997,4304572,1041_-1,102971,Цех,7506.0,N,0.0,N,N,1700.0,5.0,1.0,0.0,2.0,94.09,0.0,Down jacket,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,0
5999998,6550634,1018_-1,102152,Цех,7506.0,N,0.0,N,N,269.0,5.0,1.0,0.0,2.0,0.00,0.0,0,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000000 entries, 0 to 5999999
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   id                             int64  
 1   oper_type + oper_attr          object 
 2   index_oper                     object 
 3   type                           object 
 4   priority                       float64
 5   is_privatecategory             object 
 6   class                          float64
 7   is_in_yandex                   object 
 8   is_return                      object 
 9   weight                         float64
 10  mailtype                       float64
 11  mailctg                        float64
 12  mailrank                       float64
 13  directctg                      float64
 14  transport_pay                  float64
 15  postmark                       float64
 16  name_mfi                       object 
 17  weight_mfi                     float64
 18  pr

## Добавим посылкам отдельный признак регион.

In [5]:
df['area']=df['index_oper'].str[0:3]
le_area  = preprocessing.LabelEncoder()
le_area.fit(df["area"])
df["area"]=le_area.transform(df["area"])
df["area"]

0          182
1            3
2          175
3           93
4          183
          ... 
5999995      3
5999996     74
5999997      3
5999998      3
5999999     22
Name: area, Length: 6000000, dtype: int64

## Добавим регионам общий обьем посылок который через них проходит.

In [6]:
data_total_area=pd.DataFrame(df.groupby('area')['total_qty_over_index'].sum()).rename(columns = {'total_qty_over_index':'total_qty_over_index_area'})
data_total_area.to_csv('data_total_area.csv')
df = pd.merge(df,data_total_area, on="area", how='left')


## Считаем потерянные в каждом регионе посылки.

In [7]:
data_failed=pd.DataFrame(df.groupby('area')['label'].sum()).rename(columns = {'label':'sum_label'})
data_failed.to_csv('data_failed.csv')
df = pd.merge(df,data_failed, on="area", how='left')

## Процент потерянных

In [8]:
df['percent_loss'] =df['sum_label']/df['total_qty_over_index_area'] 

## Cделаем регион котигориальным.

In [9]:
df["priority"]

0          7503.0
1          7503.0
2          7503.0
3          7503.0
4          7503.0
            ...  
5999995    7503.0
5999996    7503.0
5999997    7506.0
5999998    7506.0
5999999    7503.0
Name: priority, Length: 6000000, dtype: float64

# В этом регионе посылки теряються всреднем в 10 раз чаще.

## Переведем остальные переменные в котигориальные.

In [10]:
le_priority  = preprocessing.LabelEncoder()
le_priority.fit(df["priority"])
df["priority"]=le_priority.transform(df["priority"])
#df["priority"] = pd.Categorical(df["priority"])
#df["priority"].astype('category').cat.codes
#df["priority"] = df["priority"].cat.codes
df["priority"]

0          1
1          1
2          1
3          1
4          1
          ..
5999995    1
5999996    1
5999997    3
5999998    3
5999999    1
Name: priority, Length: 6000000, dtype: int64

In [11]:
df["is_in_yandex"]

0          Y
1          N
2          Y
3          Y
4          Y
          ..
5999995    N
5999996    Y
5999997    N
5999998    N
5999999    Y
Name: is_in_yandex, Length: 6000000, dtype: object

In [12]:
le_is_in_yandex   = preprocessing.LabelEncoder()
le_is_in_yandex.fit(df["is_in_yandex"])
df["is_in_yandex"]=le_is_in_yandex.transform(df["is_in_yandex"])


#df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
#df["is_in_yandex"].astype('category').cat.codes
#df["is_in_yandex"] = df["is_in_yandex"].cat.codes
df["is_in_yandex"]

0          2
1          1
2          2
3          2
4          2
          ..
5999995    1
5999996    2
5999997    1
5999998    1
5999999    2
Name: is_in_yandex, Length: 6000000, dtype: int64

In [13]:
df["is_return"]

0          N
1          N
2          N
3          N
4          N
          ..
5999995    N
5999996    N
5999997    N
5999998    N
5999999    N
Name: is_return, Length: 6000000, dtype: object

In [14]:
le_is_return    = preprocessing.LabelEncoder()
le_is_return.fit(df["is_return"])
df["is_return"]=le_is_in_yandex.transform(df["is_return"])


#df["is_return"] = pd.Categorical(df["is_return"])
#df["is_return"].astype('category').cat.codes
#df["is_return"] = df["is_return"].cat.codes
df["is_return"]

0          1
1          1
2          1
3          1
4          1
          ..
5999995    1
5999996    1
5999997    1
5999998    1
5999999    1
Name: is_return, Length: 6000000, dtype: int64

In [15]:
df["oper_type + oper_attr"]

0          1043_-1
1          1023_-1
2          1018_-1
3          1019_-1
4          1020_-1
            ...   
5999995    1022_-1
5999996    1022_-1
5999997    1041_-1
5999998    1018_-1
5999999        8_2
Name: oper_type + oper_attr, Length: 6000000, dtype: object

In [16]:
le_oper_type= preprocessing.LabelEncoder()
le_oper_type.fit(df["oper_type + oper_attr"])
df["oper_type + oper_attr"]=le_oper_type.transform(df["oper_type + oper_attr"])



#df["oper_type + oper_attr"] = pd.Categorical(df["oper_type + oper_attr"])
#df["oper_type + oper_attr"].astype('category').cat.codes
#df["oper_type + oper_attr"] = df["oper_type + oper_attr"].cat.codes
df["oper_type + oper_attr"]

0          26
1          20
2          15
3          16
4          17
           ..
5999995    19
5999996    19
5999997    24
5999998    15
5999999    99
Name: oper_type + oper_attr, Length: 6000000, dtype: int64

In [17]:
le_oper_type.classes_

array(['0_-1', '1001_-1', '1002_-1', '1003_-1', '1004_-1', '1005_-1',
       '1009_-1', '1012_-1', '1013_-1', '1014_-1', '1015_-1', '1016_-1',
       '1017_-1', '1017_1000', '1017_1001', '1018_-1', '1019_-1',
       '1020_-1', '1021_-1', '1022_-1', '1023_-1', '1025_-1', '1026_-1',
       '1027_-1', '1041_-1', '1042_-1', '1043_-1', '1044_-1', '1045_-1',
       '104_-1', '10_-1', '114_-1', '11_-1', '11_0', '128_-1', '12_1',
       '12_10', '12_11', '12_2', '12_4', '12_6', '12_7', '12_8', '12_9',
       '131_-1', '14_1', '14_10', '14_1001', '14_1002', '14_16', '14_17',
       '14_18', '14_19', '14_20', '14_21', '14_6', '154_-1', '15_-1',
       '183_-1', '183_1', '184_-1', '184_1', '185_-1', '192_-1', '1_-1',
       '20_-1', '24_-1', '26_-1', '29_-1', '2_1', '2_10', '2_11', '2_2',
       '2_6', '2_8', '30_-1', '34_-1', '39_-1', '3_1', '3_10', '3_9',
       '41_-1', '41_1', '41_3', '41_4', '4_1', '4_2', '4_3', '5_1', '5_3',
       '6_1', '6_3', '6_5', '8_0', '8_1', '8_13', '8_15', '8_18', 

In [18]:
df["index_oper"]

0          628629
1          102976
2          620962
3          344964
4          629819
            ...  
5999995    102976
5999996    241963
5999997    102971
5999998    102152
5999999    140531
Name: index_oper, Length: 6000000, dtype: object

In [19]:
le_index_oper= preprocessing.LabelEncoder()
le_index_oper.fit(df["index_oper"])
df["index_oper"]=le_index_oper.transform(df["index_oper"])


#df["index_oper"] = pd.Categorical(df["index_oper"])
#df["index_oper"].astype('category').cat.codes
#df["index_oper"] = df["index_oper"].cat.codes
df["index_oper"]

0          17235
1             23
2          16331
3           6325
4          17315
           ...  
5999995       23
5999996     4697
5999997       18
5999998        6
5999999      751
Name: index_oper, Length: 6000000, dtype: int64

In [20]:
df["type"]

0          Участок
1             ММПО
2              Цех
3              Цех
4          Участок
            ...   
5999995       ММПО
5999996        Цех
5999997        Цех
5999998        Цех
5999999       СОПС
Name: type, Length: 6000000, dtype: object

In [21]:
le_type2= preprocessing.LabelEncoder()
le_type2.fit(df["type"])
df["type"]=le_type2.transform(df["type"])


#df["type"] = pd.Categorical(df["type"])
#df["type"].astype('category').cat.codes
#df["type"] = df["type"].cat.codes
df["type"]

0          18
1           4
2          19
3          19
4          18
           ..
5999995     4
5999996    19
5999997    19
5999998    19
5999999    13
Name: type, Length: 6000000, dtype: int64

In [22]:
df["is_privatecategory"]

0          N
1          N
2          N
3          N
4          N
          ..
5999995    N
5999996    N
5999997    N
5999998    N
5999999    N
Name: is_privatecategory, Length: 6000000, dtype: object

In [23]:
le_is_privatecategory= preprocessing.LabelEncoder()
le_is_privatecategory.fit(df["is_privatecategory"])
df["is_privatecategory"]=le_is_privatecategory.transform(df["is_privatecategory"])




#df["is_privatecategory"] = pd.Categorical(df["is_privatecategory"])
#df["is_privatecategory"].astype('category').cat.codes
#df["is_privatecategory"] = df["is_privatecategory"].cat.codes
df["is_privatecategory"]

0          1
1          1
2          1
3          1
4          1
          ..
5999995    1
5999996    1
5999997    1
5999998    1
5999999    1
Name: is_privatecategory, Length: 6000000, dtype: int64

In [24]:
df["name_mfi"]

0                                          Fishhook,USB
1          screen protector,Case(Q613B),case(208B40-DB)
2           Pendant Necklaces,Rings for Women,Necklaces
3                                Motorcycle Signal Lamp
4                                              backpack
                               ...                     
5999995                                       KEY CHAIN
5999996                                           Dolls
5999997                                     Down jacket
5999998                                               0
5999999                Jewelry Beads,Jewelry Components
Name: name_mfi, Length: 6000000, dtype: object

In [25]:
le_name_mfi= preprocessing.LabelEncoder()
le_name_mfi.fit(df["name_mfi"])
df["name_mfi"]=le_name_mfi.transform(df["name_mfi"])


#df["name_mfi"] = pd.Categorical(df["name_mfi"])
#df["name_mfi"].astype('category').cat.codes
#df["name_mfi"] = df["name_mfi"].cat.codes
df["name_mfi"]

0           46654
1          194484
2           92224
3           79671
4          141303
            ...  
5999995     63431
5999996     36730
5999997     37295
5999998       101
5999999     62590
Name: name_mfi, Length: 6000000, dtype: int64

In [26]:
df["weightkg"] = round(df["weight"]/1000)
df['is_wrong']=df['is_wrong_sndr_name']+df['is_wrong_rcpn_name']+df['is_wrong_phone_number']+df['is_wrong_address']
df['total_mean']=df['total_qty_over_index_and_type']/df['total_qty_over_index']

## Добавим отдельный признак для наиболее часто теряющихся типов посылок.

## Выделим выборки из финального датасета!

In [27]:
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label,area,total_qty_over_index_area,sum_label,percent_loss,weightkg,is_wrong,total_mean
0,6818780,26,17235,18,1,1,0.0,2,1,87.0,5.0,1.0,0.0,2.0,0.00,0.0,46654,41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0,182,2.748856e+11,826,3.004886e-09,0.0,0,0.093974
1,9907176,20,23,4,1,1,0.0,1,1,107.0,5.0,1.0,0.0,2.0,0.00,0.0,194484,68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,0,0.966056
2,3304275,15,16331,19,1,1,0.0,2,1,50.0,5.0,1.0,0.0,2.0,0.00,0.0,92224,56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0,175,3.939877e+12,741,1.880769e-10,0.0,1,0.122929
3,9020937,16,6325,19,1,1,0.0,2,1,416.0,5.0,1.0,0.0,2.0,35.34,0.0,79671,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0,93,6.073324e+11,345,5.680579e-10,0.0,0,0.138747
4,3082311,17,17315,18,1,1,0.0,2,1,795.0,5.0,1.0,0.0,2.0,52.52,0.0,141303,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0,183,4.110716e+10,253,6.154645e-09,1.0,0,0.073070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,19,23,4,1,1,0.0,1,1,25.0,5.0,0.0,0.0,2.0,0.00,0.0,63431,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,1,0.959104
5999996,2234489,19,4697,19,1,1,0.0,2,1,83.0,5.0,1.0,0.0,2.0,0.00,0.0,36730,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,0,74,2.701693e+11,144,5.329992e-10,0.0,0,0.122669
5999997,4304572,24,18,19,3,1,0.0,1,1,1700.0,5.0,1.0,0.0,2.0,94.09,0.0,37295,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,0,3,3.484228e+14,109350,3.138428e-10,2.0,0,0.801986
5999998,6550634,15,6,19,3,1,0.0,1,1,269.0,5.0,1.0,0.0,2.0,0.00,0.0,101,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,1,0.501738


In [28]:
df = df.fillna(0)

In [29]:
### df.to_csv('train_frame.csv', index=False)

Обьединим список не нужных строк с списком строк типа object

In [30]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label",'mailrank']))

In [31]:
col_obj

['id', 'mailrank', 'label']

In [32]:
X = df.drop(col_obj, axis = 1)
y = df["label"]

#X=X[['type', 'priority', 'class', 'dist_qty_oper_login_1',
       #'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       #'is_wrong_phone_number', 'is_wrong','area','total_qty_over_index_area','name_mfi_fail', 	'mailtype_fail', 	'oper_type_fail', 	'type_fail',]]

Xst = df.drop(col_obj, axis = 1)
yst = df["label"]

In [33]:
X

,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,area,total_qty_over_index_area,sum_label,percent_loss,weightkg,is_wrong,total_mean
0,26,17235,18,1,1,0.0,2,1,87.0,5.0,1.0,2.0,0.00,0.0,46654,41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,182,2.748856e+11,826,3.004886e-09,0.0,0,0.093974
1,20,23,4,1,1,0.0,1,1,107.0,5.0,1.0,2.0,0.00,0.0,194484,68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,0,0.966056
2,15,16331,19,1,1,0.0,2,1,50.0,5.0,1.0,2.0,0.00,0.0,92224,56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,175,3.939877e+12,741,1.880769e-10,0.0,1,0.122929
3,16,6325,19,1,1,0.0,2,1,416.0,5.0,1.0,2.0,35.34,0.0,79671,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,93,6.073324e+11,345,5.680579e-10,0.0,0,0.138747
4,17,17315,18,1,1,0.0,2,1,795.0,5.0,1.0,2.0,52.52,0.0,141303,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,183,4.110716e+10,253,6.154645e-09,1.0,0,0.073070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,19,23,4,1,1,0.0,1,1,25.0,5.0,0.0,2.0,0.00,0.0,63431,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,3,3.484228e+14,109350,3.138428e-10,0.0,1,0.959104
5999996,19,4697,19,1,1,0.0,2,1,83.0,5.0,1.0,2.0,0.00,0.0,36730,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,74,2.701693e+11,144,5.329992e-10,0.0,0,0.122669
5999997,24,18,19,3,1,0.0,1,1,1700.0,5.0,1.0,2.0,94.09,0.0,37295,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,3,3.484228e+14,109350,3.138428e-10,2.0,0,0.801986
5999998,15,6,19,3,1,0.0,1,1,269.0,5.0,1.0,2.0,0.00,0.0,101,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,1,0.501738


# Отбор признаков для обучения!

In [34]:
scaler=MinMaxScaler()
k=10 # Количество признаков для обучения!
trans=SelectKBest(chi2, k=k)
X = scaler.fit_transform(X)
X =trans.fit_transform(X, y)
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

In [35]:
filter = trans.get_support()
features = Xst.columns
 

print("Все столбцы:")
print(features)
 

print("Выбраны столбцы",k)
print(features[filter])




Все столбцы:
Index(['oper_type + oper_attr', 'index_oper', 'type', 'priority',
       'is_privatecategory', 'class', 'is_in_yandex', 'is_return', 'weight',
       'mailtype', 'mailctg', 'directctg', 'transport_pay', 'postmark',
       'name_mfi', 'weight_mfi', 'price_mfi', 'dist_qty_oper_login_1',
       'total_qty_oper_login_1', 'total_qty_oper_login_0',
       'total_qty_over_index_and_type', 'total_qty_over_index',
       'is_wrong_sndr_name', 'is_wrong_rcpn_name', 'is_wrong_phone_number',
       'is_wrong_address', 'area', 'total_qty_over_index_area', 'sum_label',
       'percent_loss', 'weightkg', 'is_wrong', 'total_mean'],
      dtype='object')
Выбраны столбцы 10
Index(['type', 'priority', 'class', 'dist_qty_oper_login_1',
       'total_qty_oper_login_0', 'total_qty_over_index_and_type',
       'is_wrong_phone_number', 'total_qty_over_index_area', 'sum_label',
       'is_wrong'],
      dtype='object')


In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y)


In [37]:
X_train.shape

(9909334, 10)

## Обучение модели

In [38]:
pipe=LogisticRegression(random_state=0)

In [39]:
pipe.fit(X_train, y_train)
# 18! 33! 38 0.826273=7

LogisticRegression(random_state=0)

## Оценка точности

In [40]:
pred =pipe.predict(X_test)

In [41]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )
print("Recall", score)

Recall 0.8198502206774609


In [42]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  pipe.predict_proba(X_test)[:, 1])

0.8839899372032716

In [43]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.96      0.67      0.79    874353
           1       0.75      0.97      0.84    874353

    accuracy                           0.82   1748706
   macro avg       0.85      0.82      0.82   1748706
weighted avg       0.85      0.82      0.82   1748706



In [44]:
print('Требуемая величина', 0.1*score+0.9*roc_auc_score(y_test,  pipe.predict_proba(X_test)[:, 1]))

Требуемая величина 0.8775759655506905


# Предсказание 

In [45]:
pred_data=pd.read_csv('test_dataset_test.csv',low_memory=False)


In [46]:
df=pred_data

In [47]:
df['area']=df['index_oper'].str[0:3]
df["area"]=le_area.transform(df["area"])
df = pd.merge(df,data_total_area, on="area", how='left')
df = pd.merge(df,data_failed, on="area", how='left')
df['percent_loss']=df['sum_label']/df['total_qty_over_index_area'] 



df["priority"]=le_priority.fit_transform(df["priority"])
df["is_in_yandex"]=le_is_in_yandex.fit_transform(df["is_in_yandex"])
df["is_return"]=le_is_in_yandex.fit_transform(df["is_return"])
df["oper_type + oper_attr"]=le_oper_type.fit_transform(df["oper_type + oper_attr"])
df["index_oper"]=le_index_oper.fit_transform(df["index_oper"])
df["type"]=le_type2.fit_transform(df["type"])
df["is_privatecategory"]=le_is_privatecategory.fit_transform(df["is_privatecategory"])
df["name_mfi"]=le_name_mfi.fit_transform(df["name_mfi"])







df["weightkg"] = round(df["weight"]/1000)
df['is_wrong']=df['is_wrong_sndr_name']+df['is_wrong_rcpn_name']+df['is_wrong_phone_number']+df['is_wrong_address']
df['total_mean']=df['total_qty_over_index_and_type']/df['total_qty_over_index']

df['name_mfi_fail']=0
for i in name_mfi_fail:
    df[df['name_mfi'] == i]['name_mfi_fail']=1
    
df['mailtype_fail']=0
for i in mailtype_fail:
    df[df['mailtype'] == i]['mailtype_fail']=1
    
df['oper_type_fail']=0
for i in oper_type_fail:
    df[df['oper_type + oper_attr'] == i]['oper_type_fail']=1
    
df['type_fail']=0
for i in type_fail:
    df[df['type'] == i]['type_fail']=1

df['priority_fail']=0
for i in priority_fail:
    df[df['priority'] == i]['priority_fail']=1

In [48]:
le_oper_type.classes_

array(['0_-1', '1001_-1', '1002_-1', '1003_-1', '1004_-1', '1005_-1',
       '1009_-1', '1012_-1', '1013_-1', '1014_-1', '1015_-1', '1016_-1',
       '1017_-1', '1017_1000', '1017_1001', '1018_-1', '1019_-1',
       '1020_-1', '1021_-1', '1022_-1', '1023_-1', '1025_-1', '1026_-1',
       '1027_-1', '1041_-1', '1042_-1', '1043_-1', '1044_-1', '1045_-1',
       '104_-1', '114_-1', '11_-1', '11_0', '12_1', '12_10', '12_11',
       '12_2', '12_21', '12_4', '12_7', '12_8', '12_9', '131_-1', '14_1',
       '14_10', '14_1001', '14_16', '14_17', '14_18', '14_19', '14_20',
       '14_21', '14_5', '154_-1', '15_-1', '183_-1', '183_1', '184_-1',
       '184_1', '185_-1', '192_-1', '1_-1', '20_-1', '24_-1', '26_-1',
       '29_-1', '2_1', '2_10', '2_11', '2_6', '2_8', '30_-1', '39_-1',
       '3_1', '3_10', '3_9', '41_-1', '41_1', '41_3', '41_4', '4_1',
       '4_2', '4_3', '5_1', '5_3', '6_1', '6_3', '6_5', '8_0', '8_1',
       '8_13', '8_15', '8_18', '8_182', '8_2', '8_20', '8_22', '8_3',
      

In [49]:
df = df.fillna(0)


In [50]:
df


,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,area,total_qty_over_index_area,sum_label,percent_loss,weightkg,is_wrong,total_mean
0,7815282,90,23,4,1,1,0.0,1,0,85.0,5.0,1.0,0.0,2.0,0.00,0.0,105982,73.0,489.0,972.0,68766046.0,91123247.0,159889293.0,164927295.0,0,0,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,0,0.969453
1,8443555,94,4430,3,1,1,4.0,2,0,21.0,5.0,0.0,0.0,2.0,0.00,0.0,116968,19.0,186.0,2.0,2895.0,1545.0,4440.0,20623.0,0,1,1,0,73,4.381023e+09,382,8.719424e-08,0.0,2,0.215294
2,6352559,17,15333,3,1,1,4.0,2,0,388.0,5.0,1.0,0.0,2.0,34.01,0.0,60268,348.0,500.0,3.0,2751.0,993.0,3744.0,37817.0,0,0,0,0,173,1.048056e+10,235,2.242246e-08,0.0,0,0.099003
3,4921420,11,16,15,1,1,0.0,1,0,1203.0,5.0,1.0,0.0,2.0,83.72,0.0,94,0.0,0.0,69.0,9379609.0,2267333.0,11646942.0,11759329.0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,1.0,1,0.990443
4,1420440,15,18,19,3,1,0.0,1,0,956.0,5.0,1.0,0.0,2.0,14.95,0.0,127073,946.0,1267.0,181.0,76138189.0,8747.0,76146936.0,84110459.0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,1.0,1,0.905321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999995,9030800,14,17999,3,1,1,4.0,2,0,53.0,5.0,1.0,0.0,2.0,0.00,0.0,192273,61.0,262.0,3.0,4297.0,1803.0,6100.0,64613.0,0,0,0,0,200,7.201455e+10,231,3.207685e-09,0.0,0,0.094408
3999996,6275048,20,23,4,1,1,0.0,1,0,286.0,5.0,1.0,0.0,2.0,32.05,0.0,94,0.0,0.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,0.0,1,0.966056
3999997,4701757,19,17,15,1,1,0.0,1,0,733.0,5.0,1.0,0.0,2.0,49.98,0.0,70072,662.0,300.0,106.0,17348074.0,3469387.0,20817461.0,20821109.0,0,1,0,0,3,3.484228e+14,109350,3.138428e-10,1.0,1,0.999825
3999998,611153,72,29,18,1,1,0.0,1,0,265.0,5.0,1.0,0.0,2.0,0.00,0.0,36312,247.0,500.0,2.0,15.0,13673720.0,13673735.0,14041298.0,0,0,0,0,5,8.731842e+11,600,6.871402e-10,0.0,0,0.973823


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000000 entries, 0 to 3999999
Data columns (total 35 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   id                             int64  
 1   oper_type + oper_attr          int64  
 2   index_oper                     int64  
 3   type                           int64  
 4   priority                       int64  
 5   is_privatecategory             int64  
 6   class                          float64
 7   is_in_yandex                   int64  
 8   is_return                      int64  
 9   weight                         float64
 10  mailtype                       float64
 11  mailctg                        float64
 12  mailrank                       float64
 13  directctg                      float64
 14  transport_pay                  float64
 15  postmark                       float64
 16  name_mfi                       int64  
 17  weight_mfi                     float64
 18  pr

In [52]:
df=df.drop(['id','mailrank'], axis = 1)

In [53]:
df = scaler.transform(df)
df =trans.transform(df)

In [54]:
pred_data['label']=pipe.predict(df)

In [55]:
pred_data[['id','label']].to_csv('bestmodel.csv', index=False)

### 